In [43]:
import numpy as np
import math
from scipy.sparse import lil_matrix, csr_matrix

In [44]:
DAMPING_FACTOR = 0.85  # Damping factor, typically set to 0.85
EPSILON = 0.00000001  # Convergence threshold
MAX_ITERATIONS = 100  # Limit on the number of iterations for convergence

In [45]:
# Function to calculate the absolute difference between two numbers
def abs_diff(a, b):
    return abs(a - b)

In [46]:
def initialize_auth(initialval, num_vertices):
    if initialval == 0:
        return np.zeros(num_vertices, dtype=np.float32)
    elif initialval == 1:
        return np.ones(num_vertices, dtype=np.float32)
    elif initialval == -1:
        return np.full(num_vertices, 1.0 / num_vertices, dtype=np.float32)
    elif initialval == -2:
        return np.full(num_vertices, 1.0 / math.sqrt(num_vertices), dtype=np.float32)

In [47]:
# Function to load graph from file as a sparse matrix
def load_graph(filename, initialval):
    with open(filename, "r") as file:
        next(file)  # Skip first line (header)
        edges = [tuple(map(int, line.strip().split())) for line in file]
    
    max_index = max(max(u, v) for u, v in edges)
    num_vertices = max_index + 1
    adj_matrix = csr_matrix((np.ones(len(edges)), zip(*edges)), shape=(num_vertices, num_vertices), dtype=np.float32)

    # Initialize rank vector based on initialval
    rank = initialize_auth(initialval, num_vertices)
    return adj_matrix, rank

In [48]:
# PageRank calculation using matrix-vector multiplication and handling dangling nodes
def page_rank(filename, initialval):
    adj_matrix, rank = load_graph(filename, initialval)
    num_vertices = len(rank)
    teleport = (1 - DAMPING_FACTOR) / num_vertices  # Constant teleportation factor
    
    out_degree = adj_matrix.sum(axis=0).A1  # Out-degree of each node (sum of columns)
    dangling_nodes = (out_degree == 0)  # Identify dangling nodes

    for iteration in range(MAX_ITERATIONS):
        # Handle dangling nodes by adding their rank evenly to all nodes
        dangling_rank = DAMPING_FACTOR * rank[dangling_nodes].sum() / num_vertices
        
        # Calculate new rank values using the PageRank formula
        new_rank = teleport + dangling_rank + DAMPING_FACTOR * adj_matrix.dot(rank / np.where(out_degree == 0, 1, out_degree))

        # Check for convergence
        if np.linalg.norm(new_rank - rank, ord=1) < EPSILON:
            print(f"Converged after {iteration + 1} iterations")
            break

        rank = new_rank

    # Display the final PageRank scores
    print("PageRank scores (showing first 10 nodes):")
    for i, score in enumerate(rank[:10]):  # Display only the first 10 nodes for brevity
        print(f"Page {i + 1}: {score:.6f}")

In [49]:
# Calculate and display the PageRank scores
filename = "web-Google.txt"  # Replace with your file path
initialval = -1  # Choose initialization value as needed
page_rank(filename, initialval)

Converged after 85 iterations
PageRank scores (showing first 10 nodes):
Page 1: 0.000000
Page 2: 0.000000
Page 3: 0.000001
Page 4: 0.000000
Page 5: 0.000002
Page 6: 0.000004
Page 7: 0.000001
Page 8: 0.000004
Page 9: 0.000013
Page 10: 0.000001
